# Importar Librerias

In [ ]:
import pygad
import time
import math
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
from datetime import datetime
from pandas.tseries.frequencies import to_offset
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn import set_config 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from tensorflow.keras.models import Sequential
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop
from keras.callbacks import EarlyStopping
################################################################################################################
physical_cpu = tf.config.list_physical_devices('CPU')
print("CPU's Disponibles: " + str(len(physical_cpu)))
for cpu in physical_cpu:
    print(cpu)
    
physical_gpu = tf.config.list_physical_devices('GPU')
print("GPU's Disponibles: " + str(len(physical_gpu)))
for gpu in physical_gpu:
    print(gpu)
################################################################################################################
best_mse = 0
best_mape = 0
best_smape = 0

# Preprocesar Serie

In [ ]:
from scipy.sparse.linalg import svds
import pandas as pd
########Control###########
file_name = "INSTANCIAS/CDMX.csv"
original_seasonal_length= 365
seasonal_length = 52 #representa la duración de un periodo en mi serie de tiempo
percentage_train = 0.7
percentage_train_val = 0.7

#Lectura de los datos
column_names = ["Date", "Temp"]
dataset_path = file_name
raw_dataset = pd.read_csv(dataset_path, names = column_names, na_values = "?", comment = "\t", sep=",", skipinitialspace = True)
dataset = raw_dataset.copy()
column_names.remove("Date") 
dataset["Date"] = dataset.Date \
                             .map(lambda x : datetime.strptime(str(x), "%d/%m/%Y"))
#reindexación y eliminación de columnas basura
dataset.index = dataset.Date
dataset.pop("Date")

#Interpolar los datos faltantes
dataset.interpolate(inplace=True, method = "time")
#Reducción del ruido
data = np.array(dataset)
#Proceso de hankelización 
ancho_hankel = original_seasonal_length   #Un periodo estacional
matrix = []
index = 0
for i in range(len(data)-ancho_hankel):
    fila = data[index:index+ancho_hankel]
    matrix.append(fila)
    index +=1
matrix = np.array(matrix)
matrix = np.reshape(matrix,(matrix.shape[0],matrix.shape[1]))
#Descomposición en valores singulares
num_components = 20
u, s, v = svds(matrix, k = num_components)
sdiag = np.diag(s)
partial = np.matmul(u, sdiag)
reconstruida = np.matmul(partial,v)
#Deshankelización
nueva_serie = []
state = True
for i in reconstruida:
    if state:
        for j in i:
            nueva_serie.append(j)
        state = False
    else:
        nueva_serie.append(i[len(i)-1])
nueva_serie.append(i[len(i)-1])
#print(nueva_serie)
#Normalización
nueva_serie = ((nueva_serie-min(nueva_serie))/(max(nueva_serie)-min(nueva_serie)))
#Reindexado
array = np.array(nueva_serie)
index_values = dataset.index
#array = array[:len(index_values)]
dataset = pd.DataFrame(data = array, 
                      index = index_values,
                      columns = column_names)
logic = {'Temp'  : 'mean'}
dataset = dataset.resample('W').apply(logic)
dataset.index -= to_offset("6D")
#Interpolar los datos faltantes
dataset.interpolate(inplace=True, method = "time")

decompose = seasonal_decompose(dataset, model = "add", extrapolate_trend='freq',  two_sided=True, period = seasonal_length)
print("-------------------------------------------------------------------------------------------------------------------")
print(decompose.trend)
np.savetxt('datos.txt', decompose.trend)
print("-------------------------------------------------------------------------------------------------------------------")
### La serie ya esta filtrada, ahora se divide en 3(train, validation, test)
percentage_train_split = int(len(dataset)*percentage_train)
test = dataset[percentage_train_split:]
test.index = [pd.Timestamp(d) for d in test.index]
train_temp = dataset[:percentage_train_split]
train_temp.index = [pd.Timestamp(d) for d in train_temp.index]
percentage_val_split = int(len(train_temp)*percentage_train_val)
validation = train_temp[percentage_val_split:]
validation.index = [pd.Timestamp(d) for d in validation.index]
train = dataset[:percentage_val_split]
train.index = [pd.Timestamp(d) for d in train.index]
#Descompuestos los 3 sets, se debe descomponer cada uno de ellos en tendencia, estacionalidad y ruido
decompose_result_train = seasonal_decompose(train,model="add", extrapolate_trend='freq',  two_sided=True, period = seasonal_length)
decompose_result_val = seasonal_decompose(validation,model="add", extrapolate_trend='freq',  two_sided=True, period = seasonal_length)
decompose_result_test = seasonal_decompose(test,model="add", extrapolate_trend='freq',  two_sided=True, period = seasonal_length)
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.plot(decompose_result_val.observed, label='Observado')
plt.legend(loc='upper left')
plt.title('Serie Temporal Observada')

plt.subplot(4, 1, 2)
plt.plot(decompose_result_val.trend, label='Tendencia')
plt.legend(loc='upper left')
plt.title('Tendencia')

plt.subplot(4, 1, 3)
plt.plot(decompose_result_val.seasonal, label='Estacionalidad')
plt.legend(loc='upper left')
plt.title('Estacionalidad')

plt.subplot(4, 1, 4)
plt.plot(decompose_result_val.resid, label='Residuos')
plt.legend(loc='upper left')
plt.title('Residuos')

plt.tight_layout()
plt.show()


def smape(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred)) / 2))



##3 bloques descompuestos en 3 componentes
train_trend = decompose_result_train.trend
train_season =decompose_result_train.seasonal
train_resid = decompose_result_train.resid
val_trend = decompose_result_val.trend
val_season =decompose_result_val.seasonal
val_resid = decompose_result_val.resid
test_trend = decompose_result_test.trend
test_season =decompose_result_test.seasonal
test_resid = decompose_result_test.resid
print("done!")

# Regresores

## Random Forest

In [ ]:
rf_trend_model = None
rf_season_model = None
rf_resid_model = None

def randomForestTrain():
    global rf_trend_model, rf_season_model, rf_resid_model
    global train_trend, train_season, train_resid
    train_tren = pd.DataFrame(train_trend).to_numpy()
    train_seaso = pd.DataFrame(train_season).to_numpy()
    train_resi = pd.DataFrame(train_resid).to_numpy()
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    rf_trend_char = []
    index = 0
    for i in range(len(train_tren)-ancho_hankel):
        fila = train_tren[index:index+ancho_hankel]
        rf_trend_char.append(fila)
        index +=1
    rf_trend_char = np.array(rf_trend_char)
    rf_trend_char = np.reshape(rf_trend_char,(rf_trend_char.shape[0],rf_trend_char.shape[1]))
    rf_trend = np.array(train_tren).reshape(-1,1)
    rf_trend = np.delete(rf_trend, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    rf_season_char = []
    index = 0
    for i in range(len(train_seaso)-ancho_hankel):
        fila = train_seaso[index:index+ancho_hankel]
        rf_season_char.append(fila)
        index +=1
    rf_season_char = np.array(rf_season_char)
    rf_season_char = np.reshape(rf_season_char,(rf_season_char.shape[0],rf_season_char.shape[1]))
    rf_season = np.array(train_seaso).reshape(-1,1)
    rf_season = np.delete(rf_season, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    rf_resid_char = []
    index = 0
    for i in range(len(train_resi)-ancho_hankel):
        fila = train_resi[index:index+ancho_hankel]
        rf_resid_char.append(fila)
        index +=1
    rf_resid_char = np.array(rf_resid_char)
    rf_resid_char = np.reshape(rf_resid_char,(rf_resid_char.shape[0],rf_resid_char.shape[1]))
    rf_resid = np.array(train_resi).reshape(-1,1)
    rf_resid = np.delete(rf_resid, range(1,ancho_hankel+1))
    #########################################################################################
    set_config(print_changed_only=False) 
    
    if True:
        rf_trend_model = RandomForestRegressor(bootstrap=True, 
                                               criterion='squared_error',  # squared_error para una tendencia suave
                                               max_depth=None, 
                                               max_features=1,  # 'auto' suele funcionar bien para muchos casos
                                               n_estimators=10000,  # empezar con un número menor de estimadores
                                               min_samples_split=2,
                                               min_samples_leaf=1,
                                               n_jobs=-1, 
                                               random_state=42,  # Para reproducibilidad
                                               verbose=0)
        rf_season_model = RandomForestRegressor(bootstrap=True, 
                                                criterion='squared_error', 
                                                max_depth=None, 
                                                max_features=1,  
                                                n_estimators=10000,  
                                                min_samples_split=2,
                                                min_samples_leaf=1,
                                                n_jobs=-1, 
                                                random_state=42,  
                                                verbose=0)
        rf_resid_model = RandomForestRegressor(bootstrap=True, 
                                               criterion='squared_error', 
                                               max_depth=None, 
                                               max_features=1,  
                                               n_estimators=10000,  
                                               min_samples_split=2,
                                               min_samples_leaf=1,
                                               n_jobs=-1, 
                                               random_state=42,  
                                               verbose=0)

    rf_trend_model = rf_trend_model.fit(rf_trend_char, rf_trend.ravel())
    rf_season_model = rf_season_model.fit(rf_season_char, rf_season.ravel()) 
    rf_resid_model = rf_resid_model.fit(rf_resid_char, rf_resid.ravel())
    
    global val_trend, val_season, val_resid
    val_tren = pd.DataFrame(val_trend).to_numpy()
    val_seaso = pd.DataFrame(val_season).to_numpy()
    val_resi = pd.DataFrame(val_resid).to_numpy()
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    rf_trend_char = []
    index = 0
    for i in range(len(val_tren)-ancho_hankel):
        fila = val_tren[index:index+ancho_hankel]
        rf_trend_char.append(fila)
        index +=1
    rf_trend_char = np.array(rf_trend_char)
    rf_trend_char = np.reshape(rf_trend_char,(rf_trend_char.shape[0],rf_trend_char.shape[1]))
    rf_trend = np.array(val_tren).reshape(-1,1)
    rf_trend = np.delete(rf_trend, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    rf_season_char = []
    index = 0
    for i in range(len(val_seaso)-ancho_hankel):
        fila = val_seaso[index:index+ancho_hankel]
        rf_season_char.append(fila)
        index +=1
    rf_season_char = np.array(rf_season_char)
    rf_season_char = np.reshape(rf_season_char,(rf_season_char.shape[0],rf_season_char.shape[1]))
    rf_season = np.array(val_seaso).reshape(-1,1)
    rf_season = np.delete(rf_season, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    rf_resid_char = []
    index = 0
    for i in range(len(val_resi)-ancho_hankel):
        fila = val_resi[index:index+ancho_hankel]
        rf_resid_char.append(fila)
        index +=1
    rf_resid_char = np.array(rf_resid_char)
    rf_resid_char = np.reshape(rf_resid_char,(rf_resid_char.shape[0],rf_resid_char.shape[1]))
    rf_resid = np.array(val_resi).reshape(-1,1)
    rf_resid = np.delete(rf_resid, range(1,ancho_hankel+1))
    #########################################################################################
    
    #Entrenados los modelos se evalua su desempeño con el conjunto de validación

    rf_trend_predict = rf_trend_model.predict(rf_trend_char)
    rf_season_predict = rf_season_model.predict(rf_season_char)
    rf_resid_predict = rf_resid_model.predict(rf_resid_char)
    print("====================Random Forest Values====================")
    #global mse_r
    smape_rf_trend = smape(rf_trend, rf_trend_predict)
    smape_rf_season = smape(rf_season, rf_season_predict)
    smape_rf_resid = smape(rf_resid, rf_resid_predict)
    mse_rf_trend = mean_squared_error(rf_trend, rf_trend_predict)
    mape_rf_trend = mean_absolute_percentage_error(rf_trend, rf_trend_predict)
    mse_rf_season = mean_squared_error(rf_season, rf_season_predict)
    mape_rf_season = mean_absolute_percentage_error(rf_season, rf_season_predict)
    mse_rf_resid = mean_squared_error(rf_resid, rf_resid_predict)
    mape_rf_resid = mean_absolute_percentage_error(rf_resid, rf_resid_predict)
    print("trend rf\t  MSE: " + str(mse_rf_trend) + "\t MAPE: "+ str(mape_rf_trend) + "\t sMAPE: "+ str(smape_rf_trend) )
    print("season rf\t  MSE: " + str(mse_rf_season) + "\t MAPE: "+ str(mape_rf_season)+ "\t sMAPE: "+ str(smape_rf_season))
    print("resid rf\t  MSE: " + str(mse_rf_resid) + "\t MAPE: "+ str(mape_rf_resid)+ "\t sMAPE: "+ str(smape_rf_resid))
    prueba = rf_trend_predict + rf_season_predict + rf_resid_predict
    prueba_original = rf_trend + rf_season + rf_resid
    print("Mape gral: ", mean_absolute_percentage_error(prueba, prueba_original))
    print("sMape gral: ", smape(prueba, prueba_original))
    print("MSE gral: ", mean_absolute_percentage_error(prueba, prueba_original))
    plt.figure(figsize=(18, 8))
    plt.plot(prueba, color = "gray")
    plt.plot(prueba_original, color = "green")
    f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18,8),
                             sharex=True)
    f.suptitle("Decomposition-Based Random Forest Forecasting")
    
    ax1.set_title('Trend component')
    ax1.plot(rf_trend, color = "green", label="observed")
    ax1.plot(rf_trend_predict, color = "gray", label="estimated")
    ax1.set(ylabel="Temperature trend")
    ax1.set(xlabel="Periods")
    f.legend()
    
    ax2.set_title('Seasonal component')
    ax2.plot(rf_season, color = "green", label="observed")
    ax2.plot(rf_season_predict, color = "gray", label="estimated")
    ax2.set(ylabel="Temperature seasonal")
    ax2.set(xlabel="Periods")
    
    ax3.set_title('Noise component')
    ax3.plot(rf_resid, color = "green", label="observed")
    ax3.plot(rf_resid_predict, color = "gray", label="estimated")
    ax3.set(xlabel="Periods")
    ax3.set(ylabel="Noise")
    f.tight_layout()
    plt.show()
    
    print("====================Done Random Forest====================\n")
    return mape_rf_trend, mape_rf_season, mape_rf_resid


#  LSTM

In [ ]:
lstm_model_trend = None
lstm_model_season = None
lstm_model_resid = None

def lstmTrain():
    global train_trend, train_season, train_resid
    train_tren = pd.DataFrame(train_trend).to_numpy()
    train_seaso = pd.DataFrame(train_season).to_numpy()
    train_resi = pd.DataFrame(train_resid).to_numpy()
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    lstm_trend_char = []
    index = 0
    for i in range(len(train_tren)-ancho_hankel):
        fila = train_tren[index:index+ancho_hankel]
        lstm_trend_char.append(fila)
        index +=1
    lstm_trend_char = np.array(lstm_trend_char)
    lstm_trend_char = np.reshape(lstm_trend_char,(lstm_trend_char.shape[0],lstm_trend_char.shape[1]))
    lstm_trend = np.array(train_tren).reshape(-1,1)
    lstm_trend = np.delete(lstm_trend, range(1,ancho_hankel+1))
    #########################################################################################
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    lstm_season_char = []
    index = 0
    for i in range(len(train_seaso)-ancho_hankel):
        fila = train_seaso[index:index+ancho_hankel]
        lstm_season_char.append(fila)
        index +=1
    lstm_season_char = np.array(lstm_season_char)
    lstm_season_char = np.reshape(lstm_season_char,(lstm_season_char.shape[0],lstm_season_char.shape[1]))
    lstm_season = np.array(train_seaso).reshape(-1,1)
    lstm_season = np.delete(lstm_season, range(1,ancho_hankel+1))
    #########################################################################################
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    lstm_resid_char = []
    index = 0
    for i in range(len(train_resi)-ancho_hankel):
        fila = train_resi[index:index+ancho_hankel]
        lstm_resid_char.append(fila)
        index +=1
    lstm_resid_char = np.array(lstm_resid_char)
    lstm_resid_char = np.reshape(lstm_resid_char,(lstm_resid_char.shape[0],lstm_resid_char.shape[1]))
    lstm_resid = np.array(train_resi).reshape(-1,1)
    lstm_resid = np.delete(lstm_resid, range(1,ancho_hankel+1))
    ############################################################################################################
    ######################################################################################################################
    global lstm_model_trend, lstm_model_season, lstm_model_resid
    ############################################################################################################
    lstm_model_trend = Sequential()
    lstm_model_trend.add(LSTM(12, input_shape=(12,1), return_sequences = False))
    #lstm_model_trend.add(LSTM (15, return_sequences = False))
    #lstm_model_trend.add(LSTM (15, return_sequences = True))
    #lstm_model_trend.add (LSTM (15, return_sequences = False)) #En esta capa se estable que el retorno de secuencias es false ya que pasa la información directamente a la red MLP  
    lstm_model_trend.add(Dense(100, activation = "tanh"))
    lstm_model_trend.add(Dense(1, activation = "linear"))
    lstm_model_trend.compile(loss="mape", optimizer=Adam(learning_rate=0.001), metrics=["mape","mse"])
    #lstm_model_trend.summary()
    ############################################################################################################
    lstm_model_season = Sequential()
    lstm_model_season.add(LSTM(24, input_shape=(12,1), return_sequences = False))
    #lstm_model_season.add(LSTM (20, return_sequences = True))
    #lstm_model_season.add(LSTM (20, return_sequences = False))
    #lstm_model_season.add(LSTM (15, return_sequences = True))
    #lstm_model_season.add (LSTM (15, return_sequences = False)) #En esta capa se estable que el retorno de secuencias es false ya que pasa la información directamente a la red MLP  
    #lstm_model_season.add(Dense(10, activation = "tanh"))
    lstm_model_season.add(Dense(100, activation = "tanh"))
    lstm_model_season.add(Dropout(0.2))
    lstm_model_season.add(Dense(100, activation = "tanh"))
    lstm_model_season.add(Dropout(0.2))
    lstm_model_season.add(Dense(100, activation = "tanh"))
    lstm_model_season.add(Dropout(0.2))
    lstm_model_season.add(Dense(100, activation = "tanh"))
    lstm_model_season.add(Dropout(0.2))
    lstm_model_season.add(Dense(1, activation = "linear"))
    lstm_model_season.compile(loss="mape", optimizer=Adam(learning_rate=0.001), metrics=["mape","mse"])
    #lstm_model_season.summary()
    ############################################################################################################
    lstm_model_resid = Sequential()
    lstm_model_resid.add(LSTM(36, input_shape=(12,1), return_sequences = False))
    #lstm_model_resid.add(LSTM (15, return_sequences = True))
    #lstm_model_resid.add(LSTM (15, return_sequences = True))
    #lstm_model_resid.add (LSTM (15, return_sequences = False)) #En esta capa se estable que el retorno de secuencias es false ya que pasa la información directamente a la red MLP  
    #lstm_model_resid.add(Dense(10, activation = "tanh"))
    lstm_model_resid.add(Dense(100, activation = "tanh"))
    lstm_model_resid.add(Dropout(0.2))
    lstm_model_resid.add(Dense(100, activation = "tanh"))
    lstm_model_resid.add(Dropout(0.2))
    lstm_model_resid.add(Dense(100, activation = "tanh"))
    lstm_model_resid.add(Dropout(0.2))
    lstm_model_resid.add(Dense(100, activation = "tanh"))
    lstm_model_resid.add(Dropout(0.2))
    lstm_model_resid.add(Dense(1, activation = "linear"))
    lstm_model_resid.compile(loss="mse", optimizer=RMSprop(learning_rate=0.001), metrics=["mape","mse"])
    #lstm_model_resid.summary()
    ############################################################################################################
         #VALIDATION
    #########################################################################################
    global val_trend, val_season, val_resid
    val_tren = pd.DataFrame(val_trend).to_numpy()
    val_seaso = pd.DataFrame(val_season).to_numpy()
    val_resi = pd.DataFrame(val_resid).to_numpy()
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    lstmv_trend_char = []
    index = 0
    for i in range(len(val_tren)-ancho_hankel):
        fila = val_tren[index:index+ancho_hankel]
        lstmv_trend_char.append(fila)
        index +=1
    lstmv_trend_char = np.array(lstmv_trend_char)
    lstmv_trend_char = np.reshape(lstmv_trend_char,(lstmv_trend_char.shape[0],lstmv_trend_char.shape[1]))
    lstmv_trend = np.array(val_tren).reshape(-1,1)
    lstmv_trend = np.delete(lstmv_trend, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    lstmv_season_char = []
    index = 0
    for i in range(len(val_seaso)-ancho_hankel):
        fila = val_seaso[index:index+ancho_hankel]
        lstmv_season_char.append(fila)
        index +=1
    lstmv_season_char = np.array(lstmv_season_char)
    lstmv_season_char = np.reshape(lstmv_season_char,(lstmv_season_char.shape[0],lstmv_season_char.shape[1]))
    lstmv_season = np.array(val_seaso).reshape(-1,1)
    lstmv_season = np.delete(lstmv_season, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    lstmv_resid_char = []
    index = 0
    for i in range(len(val_resi)-ancho_hankel):
        fila = val_resi[index:index+ancho_hankel]
        lstmv_resid_char.append(fila)
        index +=1
    lstmv_resid_char = np.array(lstmv_resid_char)
    lstmv_resid_char = np.reshape(lstmv_resid_char,(lstmv_resid_char.shape[0],lstmv_resid_char.shape[1]))
    lstmv_resid = np.array(val_resi).reshape(-1,1)
    lstmv_resid = np.delete(lstmv_resid, range(1,ancho_hankel+1))
    #########################################################################################
    
    #############################################################################################################
    callback=EarlyStopping(monitor='val_loss', patience=35),
    lstm_model_trend.fit(lstm_trend_char, lstm_trend.ravel(),validation_data=(lstmv_trend_char, lstmv_trend.ravel()),shuffle=True, epochs=500,callbacks=[callback], batch_size=10, verbose=0,use_multiprocessing=True)
    lstm_model_season.fit(lstm_season_char, lstm_season.ravel(),validation_data=(lstmv_season_char, lstmv_season.ravel()), shuffle=True, epochs=500,callbacks=[callback], batch_size=10, verbose=0,use_multiprocessing=True)
    lstm_model_resid.fit(lstm_resid_char, lstm_resid.ravel(),validation_data=(lstmv_resid_char, lstmv_resid.ravel()),shuffle=True, epochs=500,callbacks=[callback], batch_size=5, verbose=0,use_multiprocessing=True)
    
    trendPredict = lstm_model_trend.predict(lstmv_trend_char, verbose = 0)
    seasonPredict = lstm_model_season.predict(lstmv_season_char, verbose = 0) 
    residPredict = lstm_model_resid.predict(lstmv_resid_char, verbose = 0) 
    
    prueba = trendPredict + seasonPredict + residPredict
    prueba_original = lstmv_trend.ravel()+lstmv_season.ravel()+lstmv_resid.ravel()
    mse_rf_trend = mean_squared_error(prueba, prueba_original)
    mape_rf_trend = mean_absolute_percentage_error(prueba, prueba_original)
    #print(" Validation values\t  MSE: " + str(mse_rf_trend) + "\t MAPE: "+ str(mape_rf_trend))
    
    
    plt.figure(figsize=(12, 8))
    print("====================LSTM Values====================")
    #global mse_rfr
    smape_lstm_trend = smape(lstmv_trend, trendPredict)
    smape_lstm_season = smape(lstmv_season, seasonPredict)
    smape_lstm_resid = smape(lstmv_resid, residPredict)
    mse_lstm_trend = mean_squared_error(lstmv_trend, trendPredict)
    mape_lstm_trend = mean_absolute_percentage_error(lstmv_trend, trendPredict)
    mse_lstm_season = mean_squared_error(lstmv_season, seasonPredict)
    mape_lstm_season = mean_absolute_percentage_error(lstmv_season, seasonPredict)
    mse_lstm_resid = mean_squared_error(lstmv_resid, residPredict)
    mape_lstm_resid = mean_absolute_percentage_error(lstmv_resid, residPredict)
    print("trend lstm\t  MSE: " + str(mse_lstm_trend) + "\t MAPE: "+ str(mape_lstm_trend) + "\t sMAPE: "+ str(smape_lstm_trend))
    print("season lstm\t  MSE: " + str(mse_lstm_season) + "\t MAPE: "+ str(mape_lstm_season) + "\t sMAPE: "+ str(smape_lstm_season))
    print("resid lstm\t  MSE: " + str(mse_lstm_resid) + "\t MAPE: "+ str(mape_lstm_resid) + "\t sMAPE: "+ str(smape_lstm_resid))
    prueba = trendPredict + seasonPredict + residPredict 
    prueba_original = lstmv_trend.ravel()+lstmv_season.ravel()+lstmv_resid.ravel()
    print("Mape gral: ", mean_absolute_percentage_error(prueba, prueba_original))
    print("sMape gral: ", smape(prueba, prueba_original))
    print("Mse gral: ", mean_squared_error(prueba, prueba_original))
    print("====================Done LSTM====================\n")
    return mape_lstm_trend, mape_lstm_season, mape_lstm_resid

## SVR 

In [ ]:
svr_trend_model = None
svr_season_model = None
svr_resid_model = None

def SVRTrain():
    global svr_trend_model, svr_season_model, svr_resid_model
    svr_trend_model = None
    svr_season_model = None
    svr_resid_model = None
    
    
    global train_trend, train_season, train_resid
    train_tren = pd.DataFrame(train_trend).to_numpy()
    train_seaso = pd.DataFrame(train_season).to_numpy()
    train_resi = pd.DataFrame(train_resid).to_numpy()
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    rf_trend_char = []
    index = 0
    for i in range(len(train_tren)-ancho_hankel):
        fila = train_tren[index:index+ancho_hankel]
        rf_trend_char.append(fila)
        index +=1
    rf_trend_char = np.array(rf_trend_char)
    rf_trend_char = np.reshape(rf_trend_char,(rf_trend_char.shape[0],rf_trend_char.shape[1]))
    rf_trend = np.array(train_tren).reshape(-1,1)
    rf_trend = np.delete(rf_trend, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    rf_season_char = []
    index = 0
    for i in range(len(train_seaso)-ancho_hankel):
        fila = train_seaso[index:index+ancho_hankel]
        rf_season_char.append(fila)
        index +=1
    rf_season_char = np.array(rf_season_char)
    rf_season_char = np.reshape(rf_season_char,(rf_season_char.shape[0],rf_season_char.shape[1]))
    rf_season = np.array(train_seaso).reshape(-1,1)
    rf_season = np.delete(rf_season, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    rf_resid_char = []
    index = 0
    for i in range(len(train_resi)-ancho_hankel):
        fila = train_resi[index:index+ancho_hankel]
        rf_resid_char.append(fila)
        index +=1
    rf_resid_char = np.array(rf_resid_char)
    rf_resid_char = np.reshape(rf_resid_char,(rf_resid_char.shape[0],rf_resid_char.shape[1]))
    rf_resid = np.array(train_resi).reshape(-1,1)
    rf_resid = np.delete(rf_resid, range(1,ancho_hankel+1))
    #########################################################################################
    #Seleciona los parametros dependiendo de la instancia
    if True:
        svr_trend_model = SVR(kernel='linear', C=1.0, epsilon=0.1)
        svr_season_model = SVR(kernel='poly', C=1.0, degree=2, epsilon=0.1, gamma='scale', verbose=0)
        svr_resid_model = SVR(kernel='rbf', C=1.0, epsilon=0.1, gamma='scale', verbose=0)  
        
    svr_trend_model.fit(rf_trend_char, rf_trend.ravel())
    svr_season_model.fit(rf_season_char, rf_season.ravel())
    svr_resid_model.fit(rf_resid_char, rf_resid.ravel())
    global val_trend, val_season, val_resid
    val_tren = pd.DataFrame(val_trend).to_numpy()
    val_seaso = pd.DataFrame(val_season).to_numpy()
    val_resi = pd.DataFrame(val_resid).to_numpy()
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    rf_trend_char = []
    index = 0
    for i in range(len(val_tren)-ancho_hankel):
        fila = val_tren[index:index+ancho_hankel]
        rf_trend_char.append(fila)
        index +=1
    rf_trend_char = np.array(rf_trend_char)
    rf_trend_char = np.reshape(rf_trend_char,(rf_trend_char.shape[0],rf_trend_char.shape[1]))
    rf_trend = np.array(val_tren).reshape(-1,1)
    rf_trend = np.delete(rf_trend, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    rf_season_char = []
    index = 0
    for i in range(len(val_seaso)-ancho_hankel):
        fila = val_seaso[index:index+ancho_hankel]
        rf_season_char.append(fila)
        index +=1
    rf_season_char = np.array(rf_season_char)
    rf_season_char = np.reshape(rf_season_char,(rf_season_char.shape[0],rf_season_char.shape[1]))
    rf_season = np.array(val_seaso).reshape(-1,1)
    rf_season = np.delete(rf_season, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    rf_resid_char = []
    index = 0
    for i in range(len(val_resi)-ancho_hankel):
        fila = val_resi[index:index+ancho_hankel]
        rf_resid_char.append(fila)
        index +=1
    rf_resid_char = np.array(rf_resid_char)
    rf_resid_char = np.reshape(rf_resid_char,(rf_resid_char.shape[0],rf_resid_char.shape[1]))
    rf_resid = np.array(val_resi).reshape(-1,1)
    rf_resid = np.delete(rf_resid, range(1,ancho_hankel+1))
    #########################################################################################
    
    print("====================SVR Values====================")
    svr_trend_predict = svr_trend_model.predict(rf_trend_char)
    svr_season_predict = svr_season_model.predict(rf_season_char)
    svr_resid_predict = svr_resid_model.predict(rf_resid_char)
    #global mse_svr
    smape_svr_trend = smape(rf_trend, svr_trend_predict)
    smape_svr_season = smape(rf_season, svr_season_predict)
    smape_svr_resid = smape(rf_resid, svr_resid_predict)
    mse_svr_trend = mean_squared_error(rf_trend, svr_trend_predict)
    mape_svr_trend = mean_absolute_percentage_error(rf_trend, svr_trend_predict)
    mse_svr_season = mean_squared_error(rf_season, svr_season_predict)
    mape_svr_season = mean_absolute_percentage_error(rf_season, svr_season_predict)
    mse_svr_resid = mean_squared_error(rf_resid, svr_resid_predict)
    mape_svr_resid = mean_absolute_percentage_error(rf_resid, svr_resid_predict)
    
    print("trend svr\t  MSE: " + str(mse_svr_trend) + "\t MAPE: "+ str(mape_svr_trend) + "\t sMAPE: "+ str(smape_svr_trend))
    print("season svr\t  MSE: " + str(mse_svr_season) + "\t MAPE: "+ str(mape_svr_season) + "\t sMAPE: "+ str(smape_svr_season))
    print("resid svr\t  MSE: " + str(mse_svr_resid) + "\t MAPE: "+ str(mape_svr_resid) + "\t sMAPE: "+ str(smape_svr_resid))
    
    prueba = svr_trend_predict + svr_season_predict + svr_resid_predict
    prueba_original = rf_trend + rf_season + rf_resid
    print("Mape gral: ", mean_absolute_percentage_error(prueba, prueba_original))
    print("sMape gral: ", smape(prueba, prueba_original))
    print("Mse gral: ", mean_squared_error(prueba, prueba_original))
    print("====================Done SVR====================\n")
    return mape_svr_trend, mape_svr_season, mape_svr_resid

## Ensamble

In [ ]:
serie = None
svr_trend_predict = None
svr_season_predict = None
svr_resid_predict = None
rf_trend_predict = None
rf_season_predict = None
rf_resid_predict = None
lstm_trend_predict = None
lstm_season_predict = None
lstm_resid_predict = None
best_value = 0
ponderacion = None


def sintonizaEnsamble(): 
    #Datos de pronostico de los modelos   
    ancho_hankel = 12   
    global svr_trend_predict, svr_season_predict, svr_resid_predict
    global rf_trend_predict, rf_season_predict, rf_resid_predict
    global lstm_trend_predict, lstm_season_predict, lstm_resid_predict
    
##################################################################################################################
##################################################################################################################
##################################################################################################################
    train_tren = pd.DataFrame(train_trend).to_numpy()
    train_seaso = pd.DataFrame(train_season).to_numpy()
    train_resi = pd.DataFrame(train_resid).to_numpy()
    val_tren = pd.DataFrame(val_trend).to_numpy()
    val_seaso = pd.DataFrame(val_season).to_numpy()
    val_resi = pd.DataFrame(val_resid).to_numpy()
    
    val_tren = np.concatenate((train_tren,val_tren), axis=0)
    val_seaso = np.concatenate((train_seaso,val_seaso), axis=0)
    val_resi = np.concatenate((train_resi,val_resi), axis=0)
    
    trend = np.array(val_tren).reshape(-1,1)
    season = np.array(val_seaso).reshape(-1,1)
    resid = np.array(val_resi).reshape(-1,1)
    #serie
    global serie
    serie = trend+season+resid
    serie = np.delete(serie, range(1,ancho_hankel+1))
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    rf_trend_char = []
    index = 0
    for i in range(len(val_tren)-ancho_hankel):
        fila = val_tren[index:index+ancho_hankel]
        rf_trend_char.append(fila)
        index +=1
    rf_trend_char = np.array(rf_trend_char)
    rf_trend_char = np.reshape(rf_trend_char,(rf_trend_char.shape[0],rf_trend_char.shape[1]))
    rf_trend = np.array(val_tren).reshape(-1,1)
    rf_trend = np.delete(rf_trend, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    rf_season_char = []
    index = 0
    for i in range(len(val_seaso)-ancho_hankel):
        fila = val_seaso[index:index+ancho_hankel]
        rf_season_char.append(fila)
        index +=1
    rf_season_char = np.array(rf_season_char)
    rf_season_char = np.reshape(rf_season_char,(rf_season_char.shape[0],rf_season_char.shape[1]))
    rf_season = np.array(val_seaso).reshape(-1,1)
    rf_season = np.delete(rf_season, range(1,ancho_hankel+1))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    rf_resid_char = []
    index = 0
    for i in range(len(val_resi)-ancho_hankel):
        fila = val_resi[index:index+ancho_hankel]
        rf_resid_char.append(fila)
        index +=1
    rf_resid_char = np.array(rf_resid_char)
    rf_resid_char = np.reshape(rf_resid_char,(rf_resid_char.shape[0],rf_resid_char.shape[1]))
    rf_resid = np.array(val_resi).reshape(-1,1)
    rf_resid = np.delete(rf_resid, range(1,ancho_hankel+1))
    #########################################################################################
    
    ####Evaluo para svr
    svr_trend_predict = svr_trend_model.predict(rf_trend_char)
    svr_season_predict = svr_season_model.predict(rf_season_char)
    svr_resid_predict = svr_resid_model.predict(rf_resid_char)

    ####Evaluo para Random Forest
    rf_trend_predict = rf_trend_model.predict(rf_trend_char)
    rf_season_predict = rf_season_model.predict(rf_season_char)
    rf_resid_predict = rf_resid_model.predict(rf_resid_char)
    
    ####Evaluo para LSTM
    lstm_trend_predict = lstm_model_trend.predict(rf_trend_char, verbose = 0)
    lstm_season_predict = lstm_model_season.predict(rf_season_char, verbose = 0) 
    lstm_resid_predict = lstm_model_resid.predict(rf_resid_char, verbose = 0) 
##################################################################################################################
##################################################################################################################
##################################################################################################################
    
    ga_instance = pygad.GA(num_generations=10000,
                       num_parents_mating=250, #Padres
                       fitness_func=ensamble_fitness,###################
                       sol_per_pop=500, #tamaño de la poblacion
                       num_genes=9,########################
                       mutation_type="adaptive",
                       mutation_probability=(0.9,0.2),
                       init_range_low=0,
                       keep_elitism=50,
                       keep_parents=50,
                       crossover_type="scattered",
                       allow_duplicate_genes=False,
                       parent_selection_type="rws",
                       gene_space = [range(-100,100),range(-100,100),range(-100,100),range(-100,100),range(-100,100),range(-100,100),range(-100,100),range(-100,100),range(-100,100)],
                       stop_criteria=["saturate_100"],
                       gene_type=int)
    print("====================Start Genetic====================")
    start = time.time()
    ga_instance.run()
    ga_instance.plot_fitness()
    
    end = time.time()
    print('Go!')
    print("Time GA: " + str(round(end - start,1)) + " seconds")
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    print("Prueba ensamble entrenamiento-validación")
    print("Mejor Solución encontrada : {solution}".format(solution=solution))
    print("Mejor MSE encontrado: " + str(1.0/best_value))
    global ponderacion
    ponderacion = solution
    build_ensamble(solution, True)
    print("====================Done Genetic====================\n")
    return solution

def build_ensamble(solution, plot):
    svr_trend= solution[0]
    svr_season= solution[1]
    svr_resid = solution[2]
    rf_trend = solution[3]
    rf_season = solution[4]
    rf_resid = solution[5]
    lstm_trend = solution[6]
    lstm_season = solution[7]
    lstm_resid = solution[8]
    #SVR
    svr_trend_p = svr_trend_predict * (svr_trend/100)
    svr_season_p = svr_season_predict * (svr_season/100)
    svr_resid_p = svr_resid_predict * (svr_resid/100)
    #RFR
    rf_trend_p = rf_trend_predict * (rf_trend/100)
    rf_season_p = rf_season_predict * (rf_season/100)
    rf_resid_p = rf_resid_predict * (rf_resid/100)
    #LSTM
    lstm_trend_p = lstm_trend_predict * (lstm_trend/100)
    lstm_season_p = lstm_season_predict * (lstm_season/100)
    lstm_resid_p = lstm_resid_predict * (lstm_resid/100)
    
    lstm_trend_p = np.array(lstm_trend_p).reshape(lstm_trend_p.shape[0])
    lstm_season_p = np.array(lstm_season_p).reshape(lstm_season_p.shape[0])
    lstm_resid_p = np.array(lstm_resid_p).reshape(lstm_resid_p.shape[0])
    
    construccion = svr_trend_p + svr_season_p + svr_resid_p
    construccion += rf_trend_p + rf_season_p + rf_resid_p
    construccion += lstm_trend_p + lstm_season_p + lstm_resid_p
    
    mse = mean_squared_error(serie, construccion)
    mape = mean_absolute_percentage_error(serie, construccion)
    smape_v = smape(serie, construccion)
    if plot:
        print("Mejor MAPE encontrado: ", mape)
        print("Mejor sMAPE encontrado: ", smape_v)
        print("Mejor MSE encontrado: ", mse)
        plt.figure(figsize=(20, 8))
        plt.plot(construccion, color = "green")
        plt.plot(serie, color = "pink")
        plt.plot()
        plt.show()
    return mape

def ensamble_fitness(ga_instance,solution, solution_idx): #Función que mide el rmse de cada variante de red neuronal
    fitness = build_ensamble(solution, False)
    #print("original -> "+ str(fitness) + " Adec: "+str(1.0/fitness))
    fitness = 1.0/(fitness)
    global best_value
    if best_value < fitness:
        best_value = fitness
        #print(best_value, end='...')
    return fitness

##  Calculo Salida 

In [ ]:
def build_output():
    svr_trend= ponderacion[0]
    svr_season= ponderacion[1]
    svr_resid = ponderacion[2]
    rf_trend = ponderacion[3]
    rf_season = ponderacion[4]
    rf_resid = ponderacion[5]
    lstm_trend = ponderacion[6]
    lstm_season = ponderacion[7]
    lstm_resid = ponderacion[8]
    
    #Datos de pronostico de los modelos   
    ancho_hankel = 12   
    global svr_trend_predict, svr_season_predict, svr_resid_predict
    global rf_trend_predict, rf_season_predict, rf_resid_predict
    global lstm_trend_predict, lstm_season_predict, lstm_resid_predict
    
    
##################################################################################################################
##################################################################################################################
##################################################################################################################
    val_tren = pd.DataFrame(test_trend).to_numpy() 
    val_seaso = pd.DataFrame(test_season).to_numpy()
    val_resi = pd.DataFrame(test_resid).to_numpy()
    
    
    trend = np.array(val_tren).reshape(-1,1)
    season = np.array(val_seaso).reshape(-1,1)
    resid = np.array(val_resi).reshape(-1,1)
    #serie
    global serie
    serie = trend+season+resid
    serie = np.delete(serie, range(1,ancho_hankel+1))
    #########################################################################################
    #Hankel Trend
    ancho_hankel = 12   #Un periodo estacional
    rf_trend_char = []
    index = 0
    for i in range(len(val_tren)-ancho_hankel):
        fila = val_tren[index:index+ancho_hankel]
        rf_trend_char.append(fila)
        index +=1
    rf_trend_char = np.array(rf_trend_char)
    rf_trend_char = np.reshape(rf_trend_char,(rf_trend_char.shape[0],rf_trend_char.shape[1]))
    #########################################################################################
    
    #########################################################################################
    #Hankel Season
    ancho_hankel = 12   #Un periodo estacional
    rf_season_char = []
    index = 0
    for i in range(len(val_seaso)-ancho_hankel):
        fila = val_seaso[index:index+ancho_hankel]
        rf_season_char.append(fila)
        index +=1
    rf_season_char = np.array(rf_season_char)
    rf_season_char = np.reshape(rf_season_char,(rf_season_char.shape[0],rf_season_char.shape[1]))
    #########################################################################################
    
    #########################################################################################
    #Hankel Resid
    ancho_hankel = 12   #Un periodo estacional
    rf_resid_char = []
    index = 0
    for i in range(len(val_resi)-ancho_hankel):
        fila = val_resi[index:index+ancho_hankel]
        rf_resid_char.append(fila)
        index +=1
    rf_resid_char = np.array(rf_resid_char)
    rf_resid_char = np.reshape(rf_resid_char,(rf_resid_char.shape[0],rf_resid_char.shape[1]))
    #########################################################################################
    
    ####Evaluo para svr
    svr_trend_predict = svr_trend_model.predict(rf_trend_char)
    svr_season_predict = svr_season_model.predict(rf_season_char)
    svr_resid_predict = svr_resid_model.predict(rf_resid_char)

    ####Evaluo para Random Forest
    rf_trend_predict = rf_trend_model.predict(rf_trend_char)
    rf_season_predict = rf_season_model.predict(rf_season_char)
    rf_resid_predict = rf_resid_model.predict(rf_resid_char)
    ####Evaluo para LSTM
    lstm_trend_predict = lstm_model_trend.predict(rf_trend_char, verbose = 0)
    lstm_season_predict = lstm_model_season.predict(rf_season_char, verbose = 0) 
    lstm_resid_predict = lstm_model_resid.predict(rf_resid_char, verbose = 0) 
    
    

    #SVR
    svr_trend_p = svr_trend_predict * (svr_trend/100)
    svr_season_p = svr_season_predict * (svr_season/100)
    svr_resid_p = svr_resid_predict * (svr_resid/100)
    #RFR
    rf_trend_p = rf_trend_predict * (rf_trend/100)
    rf_season_p = rf_season_predict * (rf_season/100)
    rf_resid_p = rf_resid_predict * (rf_resid/100)
    #LSTM
    lstm_trend_p = lstm_trend_predict * (lstm_trend/100)
    lstm_season_p = lstm_season_predict * (lstm_season/100)
    lstm_resid_p = lstm_resid_predict * (lstm_resid/100)
    
    lstm_trend_p = np.array(lstm_trend_p).reshape(lstm_trend_p.shape[0])
    lstm_season_p = np.array(lstm_season_p).reshape(lstm_season_p.shape[0])
    lstm_resid_p = np.array(lstm_resid_p).reshape(lstm_resid_p.shape[0])
    
    construccion = svr_trend_p + svr_season_p + svr_resid_p
    construccion += rf_trend_p + rf_season_p + rf_resid_p
    construccion += lstm_trend_p + lstm_season_p + lstm_resid_p
    
    mse = mean_squared_error(serie, construccion)
    smape_v = smape(serie, construccion)
    mape = mean_absolute_percentage_error(serie, construccion)
    global best_mse, best_mape, best_smape
    best_mse += mse
    best_mape += mape
    best_smape += smape_v
    print("================================================================================================")
    print("===========================     PRONOSTICO SET PRUEBA FINAL     ================================")
    print("================================================================================================")
    print("MAPE: ", mape)
    print("sMAPE: ", smape_v)
    print("MSE: ", mse)
    print("RMSE: ", (mse**0.5))
    print("================================================================================================")
    print("================================================================================================\n")
    plt.figure(figsize=(20, 8))
    plt.plot(construccion, color = "orange")
    plt.plot(serie, color = "gray")
    plt.plot()
    plt.show()

# Main

In [ ]:
import threading
mse_rf_trend = None
mse_rf_season = None
mse_rf_resid = None
mse_svr_trend = None
mse_svr_season = None
mse_svr_resid = None
mse_lstm_trend = None
mse_lstm_season = None
mse_lstm_resid = None

 
def ThreadRandomForest():
    global mse_rf_trend, mse_rf_season, mse_rf_resid
    mse_rf_trend, mse_rf_season, mse_rf_resid = randomForestTrain()
    
def ThreadLSTM():
    global mse_lstm_trend, mse_lstm_season, mse_lstm_resid
    mse_lstm_trend, mse_lstm_season, mse_lstm_resid = lstmTrain()
    
def ThreadSVR():
    global mse_svr_trend, mse_svr_season, mse_svr_resid
    mse_svr_trend, mse_svr_season, mse_svr_resid = SVRTrain()


cantidad_corridas = 30
print("Incia ejecución general")
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())
best_mse = 0
best_mape = 0
best_smape = 0
for i in range(cantidad_corridas):
    print("\nInicia ejecución ", (i+1))
    start = time.time()
    #mse_rf_trend, mse_rf_season, mse_rf_resid = randomForestTrain()
    #mse_svr_trend, mse_svr_season, mse_svr_resid = SVRTrain()
    t1 = threading.Thread(target=ThreadRandomForest)
    t2 = threading.Thread(target=ThreadSVR)
    t3 = threading.Thread(target=ThreadLSTM)
 
    t1.start()
    t2.start()
    t3.start()
    
    t1.join()
    t2.join()
    t3.join()


    #######################################################################
 
    # both threads completely executed
    print("Done!")
    end = time.time()
    print("Time process: " + str(round(end - start,1)) + " seconds")
    sintonizaEnsamble()
    build_output()
    print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    tf.keras.backend.clear_session()
global best_mse, best_mape, best_smape
best_mse = best_mse/cantidad_corridas
best_mape = best_mape/cantidad_corridas
best_smape = best_smape/cantidad_corridas
print("MSE total:  ", best_mse)
print("MAPE total: ", best_mape)
print("sMAPE total: ", best_smape)